In [1]:

import httpx
import autogen
import os
from autogen import AssistantAgent, UserProxyAgent
from docx import Document
from tavily import TavilyClient



config_listLlama = [
    {
        "model": "meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo",
        "api_key": "c9690309d76dc9d2ef93f7780b6a75e14ad9172b2f4672f59356de88f001ff0b",
        "base_url": "https://api.together.xyz/v1",
    }
]

config_listCode = [
    {
        "model": "deepseek-coder",
        "api_key": "sk-66a05afc0ef44ef787c9d0fbe67a5fb9",
        "base_url": "https://api.deepseek.com/beta",
       "max_tokens": 8192,
    }
]

config_listChat = [
    {
        "model": "deepseek-chat",
        "api_key": "sk-66a05afc0ef44ef787c9d0fbe67a5fb9",
        "base_url": "https://api.deepseek.com/beta",
    }
]

llm_configCode = {"config_list": config_listCode, "cache_seed": 42}
llm_configChat = {"config_list": config_listChat, "cache_seed": 42}
llm_configLlama = {"config_list": config_listLlama}

llm_config = {"config_list": config_listLlama}


c:\laragon\bin\python\python-3.10\lib\site-packages\pydantic\_internal\_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


In [6]:
tareas_Analista = [
    """Cuales son los requerimientos funcionales y no funcionales  e historias de usuario para el desarrollo de un sistema de
    gestion de inventarios para una pequeña empresa de confecciones y bordados llamada LYM  , entre los principales requisitos
    se debe tener ,administracion de clientes,  administracion de colegios , administracion de prendas por colegio es decir 
    cada prenda lleva asociado un colegio ,informes de ventas ,debe tener un accesso por distintos roles es decir debe 
    contar un inicio de sesion por usuario y contraseña,tambien se debe  poder hacer ventas y asociar un cliente a cada venta ,  y generar 
    reportes de las mismas , ademas de generar facturas de venta  , se debe poder ver el stock de las prendas por ultimo que todo el sistema 
    funcione en conjunto es decir  si hago una venta que eso se reste de stock""",
    """dame un modelado de la Base de datos y un script para Mysql para un sistema de gestion de inventarios para una pequeña
    empresa de confecciones y bordados llamada LYM  ten en cuenta los requerimientos del agente "Analista_de_requerimientos".""",
]

tareas_escritura = ["""Basado en el  análisis de requerimientos de el Analista_de_requerimientos, proporciona una lista detallada 
                    de los componentes del sistema debes seguir un modelo de MVC da los componentes del sistema incluyendo la lista de  
                    controladores, modelos ,vistas ademas añade una archivo index como entrada al sistema tambien añade un navbar para poder 
                    navegar entre las vistas, ten en cuenta que para el desarrollo del proyecto no se usara framework alguno
                    y debe correr en laragon"""]


In [7]:

analistaRequerimientos = autogen.AssistantAgent(
    name="Analista_de_requerimientos",
    llm_config=llm_configChat,
    system_message="""
      Eres un analista de requerimientos altamente calificado, eres experto en traducir las
      necesidades del negocio en requerimientos funcionales y no funcionales claros y comprensibles.
      Siempre consideras la usabilidad, escalabilidad y seguridad en tus análisis, asegurándote de que
      todos los aspectos críticos del sistema estén bien cubiertos , vas a escribir TERMINATE al final cuando todo esté terminado sin caracteres especiales..
        """,
)
Experto_sql = autogen.AssistantAgent(
    name="ExpertoSql",
    llm_config=llm_configLlama,
    system_message=f"""
       Eres un experto en bases de datos SQL con una profunda comprensión del diseño y modelado de bases
       de datos. Tienes experiencia en crear arquitecturas de base de datos eficientes y escalables, y eres 
       experto en escribir scripts SQL que implementan estos modelos de manera óptima. Eres meticuloso en garantizar
       la integridad de los datos, la eficiencia de las consultas y la seguridad en el manejo de la información, vas a
       escribir TERMINATE en un segmento aparte  al final cuando todo esté terminado asi tal cual sin caracteres especiales y a lo ultimo de todo.
        """,
)
Componentes = autogen.AssistantAgent(
    name="DesglosadorComponentes",
    llm_config=llm_configCode,
    system_message="""
       Eres un desarrollador Full estack especializado en php eres muy esperimentado ,pero ya no te dedicas a
       escribir codigo vas a escribir TERMINATE al final cuando todo esté terminado, sin caracteres especiales.
        """,
)

usuario = autogen.UserProxyAgent(
    name="Usuario",
    human_input_mode="NEVER",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE") or
    "**TERMINATE**" in x.get("content", "").rstrip(),
   code_execution_config=False,
)
resultados_chat = usuario.initiate_chats(
    [
        {
            "recipient": analistaRequerimientos,
            "message": tareas_Analista[0],
            "clear_history": True,
            "silent": False,
             "summary_method": "last_msg",
            
        },
        {
            "recipient": Experto_sql,
            "message": tareas_Analista[1],
            
           
             
        },
        {
            "recipient": Componentes,
            "message": tareas_escritura[0],
            "summary_method": "reflection_with_llm",
            
        },
    ]
)



********************************************************************************
Starting a new chat....

********************************************************************************
Usuario (to Analista_de_requerimientos):

Cuales son los requerimientos funcionales y no funcionales  e historias de usuario para el desarrollo de un sistema de gestion de inventarios para una pequeña empresa de confecciones y bordados llamada LYM Confecciones , entre los principales requisitos se debe tener ,Registro y salidas de inventario , administracion de colegios , administracion de prendas por colegio ,informes de ventas 

--------------------------------------------------------------------------------
Analista_de_requerimientos (to Usuario):

### Requerimientos Funcionales

1. **Registro y Salidas de Inventario**
   - **RF-1.1** El sistema debe permitir el registro de productos (telas, hilos, botones, etc.) en el inventario.
   - **RF-1.2** El sistema debe permitir la salida de productos del 

In [8]:
respuestaRequerimientos = resultados_chat[0].chat_history[-1]["content"] if resultados_chat[0].chat_history else "No hay respuesta disponible"
respuestaSql = resultados_chat[1].chat_history[-1]["content"] if resultados_chat[1].chat_history else "No hay respuesta disponible"
respuestaComponentes = resultados_chat[2].chat_history[-1]["content"] if resultados_chat[2].chat_history else "No hay respuesta disponible"


print(f"Respuesta del Analista de Requerimientos:\n{respuestaRequerimientos}")
print(f"Respuesta del Experto SQL:\n{respuestaSql}")
print(f"Respuesta del Desglosador de Componentes:\n{respuestaComponentes}")


Respuesta del Analista de Requerimientos:
### Requerimientos Funcionales

1. **Registro y Salidas de Inventario**
   - **RF-1.1** El sistema debe permitir el registro de productos (telas, hilos, botones, etc.) en el inventario.
   - **RF-1.2** El sistema debe permitir la salida de productos del inventario cuando se utilicen en la producción.
   - **RF-1.3** El sistema debe mantener un historial de todas las entradas y salidas de inventario.
   - **RF-1.4** El sistema debe notificar automáticamente cuando un producto esté por debajo del nivel mínimo de stock.

2. **Administración de Colegios**
   - **RF-2.1** El sistema debe permitir el registro de colegios con los que LYM Confecciones tiene contratos.
   - **RF-2.2** El sistema debe permitir la edición y eliminación de información de los colegios registrados.
   - **RF-2.3** El sistema debe mantener un historial de todas las interacciones con cada colegio.

3. **Administración de Prendas por Colegio**
   - **RF-3.1** El sistema debe pe

In [9]:
task = f"""desarrolla un sistema de gestion de inventarios para una pequeña empresa de confecciones y bordados
llamada LYM  basado en los requerimientos : {respuestaRequerimientos} : y el modelado de la base de datos
: {respuestaSql},debes desarrollar el sistema completo segun los componentes que piden en {respuestaComponentes} ,
debes bridar todo el codigo de esos componentes , ten en cuenta que el proyecto debe correr en un servidor de laragon , usa bulma css para los estilos"""

# Agente Analista de Requerimientos
Desarrollador = autogen.AssistantAgent(
    name="Desarrollador",
    llm_config={"config_list": config_listCode},
    system_message=f"""
    Eres un Desarrollador experto en FUllStack experto  en php, has trabajado muchos años con él, además de que también eres un gran diseñador de UI/UX ,te encantan los buenos diseñosTe encargarás de escribir el código de toda la aplicación, completo y funcional ,ademas de indicar el paso a paso desde la creacion del proyecto desde 0 , no usaras ningun framework para este trabajo
    """,
)
DesarrolladorAux = autogen.AssistantAgent(
    name="DesarrolladorAux",
    llm_config={"config_list": config_listCode},
     system_message=f"""
    Eres un Desarrollador  experto en FUllStack experto  en php, has trabajado muchos años con él, además de que también eres un gran diseñador de UI/UX te encantan los buenos diseños Te encargarás de escribir el código de toda la aplicación, completo y funcional ,ademas de indicar el paso a paso desde la creacion del proyecto desde 0 , no usaras ningun framework para este trabajo
    """,
)
# Agente Arquitecto de Software
#ingeniero_software= autogen.AssistantAgent(
   # name="Desarrollador_Back",
    #llm_config={"config_list": config_listLlama},
    #system_message=f"""
   # system_message="Eres un Desarrollador  experto en backend en php, has trabajado muchos años con él,. Te encargarás de escribir el código de la aplicación, completo y funcional ,ademas de indicar el paso a paso desde la creacion del proyecto desde 0 , no usaras ningun framework para este trabajo
   # """,
#)

#ingeniero_software= autogen.AssistantAgent(
 #   name="ingeniero_software",
    # llm_config={"config_list": config_listLlama},
   #system_message=f"""
    #Eres un ingeniero de software . Asegúrate de que el código para cada componente cumpla con los requisitos si falta algo dicelo al desarrolador revisa si no hay codigo cortado y que segun la estructura de archivos haya escrito el codigo para todos  , tu no escribes codigo
   #""",
#)



user_proxy = autogen.UserProxyAgent(
    name="Usuario",
    human_input_mode="NEVER",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE") or
    "**TERMINATE**" in x.get("content", "").rstrip(),
   code_execution_config=False,
)
#def mensaje_reflexion(destinatario, mensajes, remitente, config):
    #print("Reflexionando...", "yellow")
    #return f"tomando el codigo de el desarrollador ,desarrolla la parte del backend del sistema teniendo en cuenta  : {respuestaComponentes} los requerimientos :{respuestaRequerimientos} y la base de datos {respuestaSql}. \n\n {destinatario.chat_messages_for_summary(remitente)[-1]['content']}"
    
    
""""  
def mensaje_reflexion(destinatario, mensajes, remitente, config):
    print("Reflexionando...", "yellow")
    return f"Reflexiona y proporciona un analisis y complemento del codigo brindado segun la lista de componentes : {respuestaComponentes} y segun los requerimientos {respuestaRequerimientos} , asegurate de que todos los componentes estan completos verifica que segun la estructura de directorios todos los archivos esten completos es decir este el codigo en su totalidad si falta algo dicelo . \n\n {destinatario.chat_messages_for_summary(remitente)[-1]['content']}"
"""
def mensaje_reflexion(destinatario, mensajes, remitente, config):
    print("Reflexionando...", "yellow")
    return f"continua el realizando el codigo exactamente donde lo dejo el otro desarrollador y completalo, ten en cuenta los requerimientos , los  componentes : {respuestaComponentes} y la base de datos {respuestaSql},ten en cuenta que el programa debe correr en laragon. \n\n {destinatario.chat_messages_for_summary(remitente)[-1]['content']}"
# El Arquitecto revisará los requerimientos del Analista
user_proxy.register_nested_chats(
    [{"recipient": DesarrolladorAux, "message": mensaje_reflexion, "summary_method": "last_msg", "max_turns": 2}],
    trigger=Desarrollador,
)

# Iniciar la conversación con el Analista de Requerimientos
res = user_proxy.initiate_chat(recipient=Desarrollador, message=task, max_turns=2, summary_method="last_msg")


Usuario (to Desarrollador):

desarrolla un sistema de gestion de inventarios para una pequeña empresa de confecciones y bordados llamada LYM basado en los requerimientos : ### Requerimientos Funcionales

1. **Registro y Salidas de Inventario**
   - **RF-1.1** El sistema debe permitir el registro de productos (telas, hilos, botones, etc.) en el inventario.
   - **RF-1.2** El sistema debe permitir la salida de productos del inventario cuando se utilicen en la producción.
   - **RF-1.3** El sistema debe mantener un historial de todas las entradas y salidas de inventario.
   - **RF-1.4** El sistema debe notificar automáticamente cuando un producto esté por debajo del nivel mínimo de stock.

2. **Administración de Colegios**
   - **RF-2.1** El sistema debe permitir el registro de colegios con los que LYM Confecciones tiene contratos.
   - **RF-2.2** El sistema debe permitir la edición y eliminación de información de los colegios registrados.
   - **RF-2.3** El sistema debe mantener un histo

2024-09-23 21:34:44,874 WARNING autogen.oai.client: Model deepseek-chat is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


Desarrollador (to Usuario):

### Implementación del Sistema de Gestión de Inventarios para LYM Confecciones

A continuación, se presenta la implementación completa del sistema de gestión de inventarios para LYM Confecciones, siguiendo la estructura MVC (Modelo-Vista-Controlador) y utilizando PHP puro sin frameworks. Además, se utilizará Bulma CSS para los estilos.

### Estructura de Carpetas

```
/lym_confecciones
    /controllers
        ColegiosController.php
        PrendasController.php
        InventarioController.php
        VentasController.php
        UsuariosController.php
    /models
        Colegio.php
        Prenda.php
        Inventario.php
        MovimientoInventario.php
        Usuario.php
        Venta.php
    /views
        /colegios
            index.php
            create.php
            edit.php
        /prendas
            index.php
            create.php
            edit.php
        /inventario
            index.php
            entrada.php
            salida.php

2024-09-23 21:37:56,776 WARNING autogen.oai.client: Model deepseek-chat is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


DesarrolladorAux (to Usuario):

### Continuación de las Vistas

#### `inventario/salida.php`

```php
<!DOCTYPE html>
<html lang="es">
<head>
    <meta charset="UTF-8">
    <title>Registrar Salida</title>
    <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/bulma/0.9.3/css/bulma.min.css">
</head>
<body>
    <?php include '../navbar.php'; ?>
    <div class="container">
        <h1 class="title">Registrar Salida</h1>
        <form action="index.php?controller=inventario&action=salida" method="post">
            <div class="field">
                <label class="label">Colegio</label>
                <div class="control">
                    <div class="select">
                        <select name="id_colegio" required>
                            <?php foreach ($colegios as $colegio): ?>
                            <option value="<?= $colegio['id_colegio'] ?>"><?= $colegio['nombre'] ?></option>
                            <?php endforeach; ?>
                        </s

2024-09-23 21:39:53,720 WARNING autogen.oai.client: Model deepseek-chat is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


DesarrolladorAux (to Usuario):

### Consideraciones Finales y Mejoras Sugeridas

#### Seguridad

1. **Validación de Entrada**: Asegúrate de validar y sanitizar todas las entradas del usuario para prevenir inyecciones SQL y otros ataques de inyección.
2. **Uso de Sesiones y Cookies**: Implementa un sistema de autenticación seguro utilizando sesiones y cookies. Asegúrate de que las sesiones tengan un tiempo de vida limitado y que las cookies estén protegidas.
3. **Encriptación de Contraseñas**: Utiliza funciones de encriptación seguras como `password_hash()` y `password_verify()` para almacenar y verificar contraseñas.
4. **Prevención de Inyecciones SQL**: Utiliza sentencias preparadas y bind parameters para todas las consultas SQL.

#### Escalabilidad

1. **Modularización**: Divide el código en módulos más pequeños y reutilizables. Esto facilitará la adición de nuevas funcionalidades en el futuro.
2. **Optimización de Consultas**: Asegúrate de que las consultas SQL sean eficientes y uti

2024-09-23 21:41:59,519 WARNING autogen.oai.client: Model deepseek-chat is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


Desarrollador (to Usuario):

### Conclusión

Este sistema de gestión de inventarios para LYM Confecciones es un buen punto de partida, pero siempre hay espacio para mejoras y optimizaciones. Siguiendo las consideraciones y mejoras sugeridas, podrás asegurar un sistema más seguro, escalable y fácil de mantener. ¡Buena suerte con tu proyecto!

### Resumen de Mejoras Sugeridas

1. **Sistema de Autenticación**: Implementa un sistema de autenticación completo para proteger las secciones del sistema que requieren permisos especiales.
2. **Sistema de Permisos**: Implementa un sistema de permisos para restringir el acceso a ciertas secciones del sistema basado en el rol del usuario.
3. **Sistema de Logs**: Implementa un sistema de logs para registrar eventos importantes en el sistema.
4. **Validación de Entrada**: Asegúrate de validar y sanitizar todas las entradas del usuario para prevenir inyecciones SQL y otros ataques de inyección.
5. **Uso de Sesiones y Cookies**: Implementa un sistema de